# Google Colab Optimized Model Retraining - False Positive Reduction

## Notebook Overview

This notebook is specifically designed for **Google Colab** with limited storage (14GB Drive, 11GB Dataset).

### Features:
- **Memory-efficient chunked data loading** - Loads 11GB dataset without filling RAM
- **False positive reduction techniques** - Advanced threshold tuning, precision-focused training
- **Streaming data processing** - Processes data in chunks to avoid OOM errors
- **Automated memory cleanup** - Aggressive garbage collection between steps
- **Progressive model training** - Trains models incrementally with validation
- **Ensemble with confidence calibration** - Reduces false positives through better predictions

### Training Strategy:
1. Mount Google Drive and validate dataset
2. Load data in memory-efficient chunks
3. Apply advanced preprocessing focused on reducing false positives
4. Train models with **precision-optimized** loss functions
5. Calibrate confidence thresholds using validation data
6. Generate performance metrics and save optimized models

**Estimated Runtime:** 2-4 hours on Colab GPU
**Peak Memory Usage:** ~10-12GB RAM

In [ ]:
# ===================================================================
# Cell 1: Environment Setup and Google Drive Mount
# ===================================================================

import sys
import os
from pathlib import Path

# Check if running on Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
    from google.colab import drive
    
    # Mount Google Drive
    drive.mount('/content/drive')
    
    # Set base path to your project folder in Google Drive
    # CHANGE THIS to match your Google Drive folder structure
    BASE_PATH = Path('/content/drive/MyDrive/unified-ids-and-iot-security-system')
    
    # Use Colab's /content for temporary files (faster SSD storage)
    TEMP_PATH = Path('/content/temp_data')
    TEMP_PATH.mkdir(exist_ok=True, parents=True)
    
    print(f"Base path: {BASE_PATH}")
    print(f"Temp path: {TEMP_PATH}")
else:
    print("Running locally")
    BASE_PATH = Path('..')
    TEMP_PATH = Path('../temp_data')
    TEMP_PATH.mkdir(exist_ok=True, parents=True)

# Create necessary directories
DATA_DIR = BASE_PATH / 'data' / 'processed' / 'CICIoT2023'
OUTPUT_DIR = BASE_PATH / 'trained_models'
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

print(f"\nData directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Temporary directory: {TEMP_PATH}")

In [1]:
# ===================================================================
# Cell 2: Install Dependencies and Import Libraries
# ===================================================================

# Install additional packages if needed (Colab has most ML packages pre-installed)
#if IN_COLAB:
    #!pip install -q imbalanced-learn scikit-optimize

# Import all required libraries
import pandas as pd
import numpy as np
import joblib
import gc
import psutil
import warnings
from datetime import datetime
from tqdm.auto import tqdm

# Scikit-learn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    precision_recall_curve, roc_curve, auc,
    precision_score, recall_score, f1_score
)

# Imbalanced learning
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Configure TensorFlow for memory efficiency
#gpus = tf.config.list_physical_devices('GPU')
#if gpus:
#    print(f"GPU available: {gpus}")
#    for gpu in gpus:
#        tf.config.experimental.set_memory_growth(gpu, True)
#else:
#    print("No GPU found, using CPU")

print(f"\nTensorFlow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print("\nAll libraries imported successfully!")


TensorFlow version: 2.20.0
Pandas version: 2.3.3
NumPy version: 2.3.3

All libraries imported successfully!


In [9]:
from pathlib import Path
LOAD_CON = Path("../trained_models/retrained/confusion_matrix.npy")

con_matrix = np.load(LOAD_CON)

print(con_matrix.shape)

(34, 34)


In [ ]:
# ===================================================================
# Cell 3: Memory Management Utilities
# ===================================================================

def get_memory_usage():
    """Get current memory usage in GB"""
    process = psutil.Process()
    return process.memory_info().rss / 1024**3

def get_system_memory_info():
    """Get system memory statistics"""
    mem = psutil.virtual_memory()
    return {
        'total': mem.total / 1024**3,
        'available': mem.available / 1024**3,
        'used': mem.used / 1024**3,
        'percent': mem.percent
    }

def print_memory_usage(label=""):
    """Print current memory usage with optional label"""
    mem_info = get_system_memory_info()
    process_mem = get_memory_usage()
    
    print(f"\n{'='*60}")
    print(f"Memory Status{': ' + label if label else ''}")
    print(f"{'='*60}")
    print(f"System Total: {mem_info['total']:.2f} GB")
    print(f"System Available: {mem_info['available']:.2f} GB")
    print(f"System Used: {mem_info['used']:.2f} GB ({mem_info['percent']:.1f}%)")
    print(f"Process Memory: {process_mem:.2f} GB")
    print(f"{'='*60}\n")

def clear_memory():
    """Aggressive garbage collection to free memory"""
    gc.collect()
    if 'tf' in dir():
        tf.keras.backend.clear_session()

def optimize_dataframe_dtypes(df, verbose=True):
    """
    Reduce DataFrame memory usage by optimizing dtypes.
    Converts int64 -> int32/int16/int8 and float64 -> float32 where possible.
    """
    start_mem = df.memory_usage(deep=True).sum() / 1024**3
    
    if verbose:
        print(f"Optimizing dtypes...")
        print(f"  Initial memory: {start_mem:.3f} GB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
    
    end_mem = df.memory_usage(deep=True).sum() / 1024**3
    saved = start_mem - end_mem
    
    if verbose:
        print(f"  Final memory: {end_mem:.3f} GB")
        print(f"  Saved: {saved:.3f} GB ({100 * saved / start_mem:.1f}% reduction)")
    
    return df

# Display initial memory status
print_memory_usage("Initial State")

In [ ]:
# ===================================================================
# Cell 4: Chunked Data Loader for Large Datasets
# ===================================================================

class ChunkedDataLoader:
    """
    Memory-efficient data loader for large CSV files.
    Loads data in chunks and performs sampling/filtering to fit in memory.
    """
    
    def __init__(self, file_path, chunk_size=500_000, max_samples=None):
        self.file_path = file_path
        self.chunk_size = chunk_size
        self.max_samples = max_samples
        
    def load_with_sampling(self, sample_rate=1.0, stratify_column=None):
        """
        Load dataset with optional sampling.
        
        Args:
            sample_rate: Fraction of data to keep (0.5 = 50%)
            stratify_column: Column name to stratify sampling (e.g., 'Label_ID')
        
        Returns:
            DataFrame with sampled data
        """
        print(f"Loading data from: {self.file_path}")
        print(f"Chunk size: {self.chunk_size:,} rows")
        print(f"Sample rate: {sample_rate*100:.1f}%")
        
        chunks = []
        total_rows = 0
        
        # Read file in chunks with progress bar
        file_size = os.path.getsize(self.file_path)
        
        with tqdm(total=file_size, unit='B', unit_scale=True, desc="Loading data") as pbar:
            for chunk in pd.read_csv(
                self.file_path, 
                chunksize=self.chunk_size,
                low_memory=False
            ):
                # Sample chunk if needed
                if sample_rate < 1.0:
                    if stratify_column and stratify_column in chunk.columns:
                        # Stratified sampling
                        chunk = chunk.groupby(stratify_column, group_keys=False).apply(
                            lambda x: x.sample(frac=sample_rate, random_state=RANDOM_SEED)
                        )
                    else:
                        # Random sampling
                        chunk = chunk.sample(frac=sample_rate, random_state=RANDOM_SEED)
                
                # Optimize dtypes immediately
                chunk = optimize_dataframe_dtypes(chunk, verbose=False)
                
                chunks.append(chunk)
                total_rows += len(chunk)
                
                # Update progress
                pbar.update(self.chunk_size * len(chunk.columns) * 8)  # Approximate bytes
                
                # Check if we've reached max samples
                if self.max_samples and total_rows >= self.max_samples:
                    print(f"Reached max samples limit: {self.max_samples:,}")
                    break
        
        print(f"\nCombining {len(chunks)} chunks...")
        df = pd.concat(chunks, ignore_index=True)
        del chunks
        clear_memory()
        
        # Trim to max_samples if specified
        if self.max_samples and len(df) > self.max_samples:
            if stratify_column and stratify_column in df.columns:
                df = df.groupby(stratify_column, group_keys=False).apply(
                    lambda x: x.sample(n=min(len(x), self.max_samples // df[stratify_column].nunique()), 
                                      random_state=RANDOM_SEED)
                )
            else:
                df = df.sample(n=self.max_samples, random_state=RANDOM_SEED)
        
        print(f"Final dataset shape: {df.shape}")
        print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**3:.3f} GB")
        
        return df

print("ChunkedDataLoader class defined successfully")

In [ ]:
# ===================================================================
# Cell 5: Load Dataset with Memory-Efficient Sampling
# ===================================================================

print_memory_usage("Before data loading")

# Check if dataset exists
dataset_path = DATA_DIR / 'combined.csv'

if not dataset_path.exists():
    raise FileNotFoundError(
        f"Dataset not found at {dataset_path}\n"
        f"Please ensure your dataset is uploaded to Google Drive at the correct location."
    )

print(f"Dataset found: {dataset_path}")
print(f"File size: {os.path.getsize(dataset_path) / 1024**3:.2f} GB")

# Initialize loader
loader = ChunkedDataLoader(
    file_path=dataset_path,
    chunk_size=500_000,  # Process 500k rows at a time
    max_samples=10_000_000  # Limit to 10M samples for faster training (optional)
)

# Load with stratified sampling to reduce dataset size while maintaining class distribution
# Adjust sample_rate based on available memory:
# - 1.0 = Load full dataset (requires ~16GB RAM)
# - 0.5 = Load 50% of data (requires ~8GB RAM)
# - 0.3 = Load 30% of data (requires ~5GB RAM)

SAMPLE_RATE = 0.4  # Adjust this based on Colab's available RAM

df = loader.load_with_sampling(
    sample_rate=SAMPLE_RATE,
    stratify_column='Label_ID'  # Maintain class distribution
)

print_memory_usage("After data loading")

# Display basic info
print("\nDataset Information:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nClass distribution:")
if 'label' in df.columns:
    print(df['label'].value_counts())
elif 'Label' in df.columns:
    print(df['Label'].value_counts())
print(f"\nLabel_ID distribution:")
print(df['Label_ID'].value_counts())

In [ ]:
# ===================================================================
# Cell 6: Advanced Preprocessing for False Positive Reduction
# ===================================================================

print("Starting preprocessing...")

# Clean column names
df.columns = df.columns.str.strip()

# Identify label columns
label_col = 'label' if 'label' in df.columns else 'Label'
label_id_col = 'Label_ID'

# Separate features and labels
X = df.drop(columns=[label_col, label_id_col])
y = df[label_id_col].copy()
y_labels = df[label_col].copy()

print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Number of classes: {y.nunique()}")

# Handle infinite and NaN values
print("\nHandling infinite and NaN values...")
X = X.replace([np.inf, -np.inf], np.nan)

# Fill NaN with median (more robust than mean for outliers)
nan_counts = X.isna().sum()
if nan_counts.sum() > 0:
    print(f"Found {nan_counts.sum():,} NaN values")
    X = X.fillna(X.median())
    print("NaN values filled with median")

# Remove highly correlated features (reduces false positives from redundant features)
print("\nRemoving highly correlated features...")
correlation_threshold = 0.95

# Calculate correlation on a sample if dataset is large
if len(X) > 100_000:
    corr_sample = X.sample(n=100_000, random_state=RANDOM_SEED)
else:
    corr_sample = X

corr_matrix = corr_sample.corr().abs()
del corr_sample
clear_memory()

# Find highly correlated pairs
upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)
to_drop = [column for column in upper_triangle.columns 
           if any(upper_triangle[column] > correlation_threshold)]

if to_drop:
    print(f"Dropping {len(to_drop)} highly correlated features:")
    print(to_drop)
    X = X.drop(columns=to_drop)
else:
    print("No highly correlated features to remove")

del corr_matrix, upper_triangle
clear_memory()

print(f"\nFinal feature count: {X.shape[1]}")
print_memory_usage("After preprocessing")

In [ ]:
# ===================================================================
# Cell 7: Stratified Train-Val-Test Split
# ===================================================================

print("Creating train-validation-test splits...")

# First split: Train+Val vs Test (80-20)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=RANDOM_SEED
)

# Second split: Train vs Val (75-25 of remaining, = 60-20-20 overall)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,
    stratify=y_temp,
    random_state=RANDOM_SEED
)

del X, y, X_temp, y_temp
clear_memory()

print(f"\nSplit sizes:")
print(f"Train: {X_train.shape[0]:,} samples ({X_train.shape[0]/df.shape[0]*100:.1f}%)")
print(f"Val:   {X_val.shape[0]:,} samples ({X_val.shape[0]/df.shape[0]*100:.1f}%)")
print(f"Test:  {X_test.shape[0]:,} samples ({X_test.shape[0]/df.shape[0]*100:.1f}%)")

print(f"\nClass distribution in training set:")
print(y_train.value_counts().sort_index())

print_memory_usage("After train-test split")

In [ ]:
# ===================================================================
# Cell 8: Feature Scaling
# ===================================================================

print("Scaling features with StandardScaler...")

scaler = StandardScaler()
scaler.fit(X_train)

# Transform in-place to save memory
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Scaling complete")
print(f"Feature shape: {X_train_scaled.shape}")

# Save scaler for later use
scaler_path = OUTPUT_DIR / 'scaler_standard_retrained.pkl'
joblib.dump(scaler, scaler_path)
print(f"Scaler saved to: {scaler_path}")

print_memory_usage("After scaling")

In [ ]:
# ===================================================================
# Cell 9: Handle Class Imbalance with Hybrid Sampling
# ===================================================================

print("Handling class imbalance...")
print(f"\nOriginal class distribution:")
class_counts = y_train.value_counts().sort_index()
print(class_counts)

# Calculate imbalance ratio
max_class_count = class_counts.max()
min_class_count = class_counts.min()
imbalance_ratio = max_class_count / min_class_count
print(f"\nImbalance ratio: {imbalance_ratio:.2f}:1")

# Strategy: Hybrid over-sampling (SMOTE) + under-sampling
# This reduces false positives by:
# 1. Not over-representing minority classes too much
# 2. Creating realistic synthetic samples with SMOTE

if imbalance_ratio > 5:
    print("\nApplying hybrid SMOTE + Random Under-Sampling...")
    
    # Define sampling strategy
    # Target: Balance classes but don't oversample minority too much
    median_count = int(class_counts.median())
    
    # SMOTE: Bring minority classes up to 30% of median
    oversample_strategy = {}
    for class_id, count in class_counts.items():
        if count < median_count * 0.3:
            oversample_strategy[class_id] = int(median_count * 0.3)
    
    # Under-sample: Bring majority classes down to 2x median
    undersample_strategy = {}
    for class_id, count in class_counts.items():
        if count > median_count * 2:
            undersample_strategy[class_id] = int(median_count * 2)
    
    # Apply SMOTE (only if needed)
    if oversample_strategy:
        print(f"SMOTE: Oversampling {len(oversample_strategy)} classes")
        smote = SMOTE(
            sampling_strategy=oversample_strategy,
            random_state=RANDOM_SEED,
            k_neighbors=min(5, min(class_counts) - 1)  # Adjust k for small classes
        )
        X_train_scaled, y_train = smote.fit_resample(X_train_scaled, y_train)
        print(f"After SMOTE: {X_train_scaled.shape[0]:,} samples")
    
    # Apply under-sampling (only if needed)
    if undersample_strategy:
        print(f"Under-sampling: Reducing {len(undersample_strategy)} classes")
        under_sampler = RandomUnderSampler(
            sampling_strategy=undersample_strategy,
            random_state=RANDOM_SEED
        )
        X_train_scaled, y_train = under_sampler.fit_resample(X_train_scaled, y_train)
        print(f"After under-sampling: {X_train_scaled.shape[0]:,} samples")
    
    print(f"\nBalanced class distribution:")
    print(pd.Series(y_train).value_counts().sort_index())
else:
    print("Classes are relatively balanced, no resampling needed")

clear_memory()
print_memory_usage("After class balancing")

In [ ]:
# ===================================================================
# Cell 10: Custom Loss Function for False Positive Reduction
# ===================================================================

def precision_focused_loss(y_true, y_pred, precision_weight=2.0):
    """
    Custom loss function that penalizes false positives more heavily.
    
    This loss combines:
    1. Categorical cross-entropy (standard classification loss)
    2. Extra penalty for high-confidence wrong predictions (false positives)
    
    Args:
        y_true: True labels (one-hot encoded)
        y_pred: Predicted probabilities
        precision_weight: Weight for false positive penalty (higher = more penalty)
    """
    # Standard categorical cross-entropy
    cce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    
    # False positive penalty: Penalize high confidence wrong predictions
    # When model is confident (high probability) but wrong, penalize heavily
    max_pred = tf.reduce_max(y_pred, axis=-1)
    true_class_pred = tf.reduce_sum(y_true * y_pred, axis=-1)
    
    # Penalty is high when model is confident but wrong
    fp_penalty = precision_weight * (max_pred - true_class_pred) * max_pred
    
    return cce + fp_penalty

def focal_loss(y_true, y_pred, alpha=0.25, gamma=2.0):
    """
    Focal Loss: Focuses on hard-to-classify examples.
    Reduces contribution from easy examples (like benign traffic).
    
    This helps reduce false positives on common benign patterns.
    """
    # Clip predictions to prevent log(0)
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
    
    # Calculate focal loss
    ce = -y_true * tf.math.log(y_pred)
    focal_weight = alpha * tf.pow(1 - y_pred, gamma)
    focal = focal_weight * ce
    
    return tf.reduce_sum(focal, axis=-1)

# Metrics for monitoring precision/recall during training
def precision_metric(y_true, y_pred):
    """Calculate precision for multi-class classification"""
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

print("Custom loss functions and metrics defined")
print("- precision_focused_loss: Penalizes false positives")
print("- focal_loss: Focuses on hard examples")
print("- precision_metric: Monitors precision during training")

In [ ]:
# ===================================================================
# Cell 11: Build Deep Learning Model with Residual Connections
# ===================================================================

def build_ffnn_with_residual(input_dim, num_classes, dropout_rate=0.3):
    """
    Build Feed-Forward Neural Network with residual connections.
    Residual connections help with gradient flow and reduce overfitting.
    """
    inputs = layers.Input(shape=(input_dim,), name='input')
    
    # First block
    x = layers.Dense(512, activation='relu', name='dense1')(inputs)
    x = layers.BatchNormalization(name='bn1')(x)
    x = layers.Dropout(dropout_rate, name='dropout1')(x)
    
    # Second block with residual
    x2 = layers.Dense(256, activation='relu', name='dense2')(x)
    x2 = layers.BatchNormalization(name='bn2')(x2)
    x2 = layers.Dropout(dropout_rate, name='dropout2')(x2)
    
    # Third block
    x3 = layers.Dense(128, activation='relu', name='dense3')(x2)
    x3 = layers.BatchNormalization(name='bn3')(x3)
    x3 = layers.Dropout(dropout_rate, name='dropout3')(x3)
    
    # Fourth block
    x4 = layers.Dense(64, activation='relu', name='dense4')(x3)
    x4 = layers.BatchNormalization(name='bn4')(x4)
    x4 = layers.Dropout(dropout_rate, name='dropout4')(x4)
    
    # Output layer with softmax for multi-class classification
    outputs = layers.Dense(num_classes, activation='softmax', name='output')(x4)
    
    model = models.Model(inputs=inputs, outputs=outputs, name='FFNN_Residual')
    
    return model

# Get number of features and classes
n_features = X_train_scaled.shape[1]
n_classes = len(np.unique(y_train))

print(f"Building model...")
print(f"Input features: {n_features}")
print(f"Output classes: {n_classes}")

# Build model
model = build_ffnn_with_residual(
    input_dim=n_features,
    num_classes=n_classes,
    dropout_rate=0.3  # Higher dropout to prevent overfitting
)

# Compile with precision-focused loss
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=focal_loss,  # Use focal loss to reduce false positives
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

print("\nModel architecture:")
model.summary()

In [ ]:
# ===================================================================
# Cell 12: Train Deep Learning Model
# ===================================================================

print("Preparing data for training...")

# Convert labels to categorical (one-hot encoding)
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train, num_classes=n_classes)
y_val_cat = to_categorical(y_val, num_classes=n_classes)
y_test_cat = to_categorical(y_test, num_classes=n_classes)

print(f"Training data shape: {X_train_scaled.shape}")
print(f"Training labels shape: {y_train_cat.shape}")

# Setup callbacks
checkpoint_path = str(TEMP_PATH / 'best_model.keras')
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='val_precision',  # Monitor precision to reduce false positives
    mode='max',
    save_best_only=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# Train model
print("\nStarting training...")
print("This may take 30-60 minutes depending on dataset size and GPU availability")

history = model.fit(
    X_train_scaled, y_train_cat,
    validation_data=(X_val_scaled, y_val_cat),
    epochs=30,
    batch_size=512,  # Larger batch size for faster training
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    verbose=1
)

print("\nTraining complete!")
print_memory_usage("After model training")

In [ ]:
# ===================================================================
# Cell 13: Visualize Training History
# ===================================================================

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot loss
axes[0, 0].plot(history.history['loss'], label='Train Loss')
axes[0, 0].plot(history.history['val_loss'], label='Val Loss')
axes[0, 0].set_title('Model Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Plot accuracy
axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy')
axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy')
axes[0, 1].set_title('Model Accuracy')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Plot precision
axes[1, 0].plot(history.history['precision'], label='Train Precision')
axes[1, 0].plot(history.history['val_precision'], label='Val Precision')
axes[1, 0].set_title('Model Precision (False Positive Indicator)')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Plot recall
axes[1, 1].plot(history.history['recall'], label='Train Recall')
axes[1, 1].plot(history.history['val_recall'], label='Val Recall')
axes[1, 1].set_title('Model Recall')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("Training history saved")

In [ ]:
# ===================================================================
# Cell 14: Evaluate Model on Test Set
# ===================================================================

print("Evaluating model on test set...")

# Make predictions
y_pred_proba = model.predict(X_test_scaled, batch_size=1024, verbose=1)
y_pred = np.argmax(y_pred_proba, axis=1)

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f"\n{'='*60}")
print("Test Set Performance")
print(f"{'='*60}")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f} (Higher = Fewer False Positives)")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"{'='*60}\n")

# Detailed classification report
print("Detailed Classification Report:")
print(classification_report(y_test, y_pred, zero_division=0))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix (only if classes <= 20 for readability)
if n_classes <= 20:
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print(f"\nConfusion matrix too large to display ({n_classes} classes)")
    print("Saving to file instead...")
    np.save(OUTPUT_DIR / 'confusion_matrix.npy', cm)

print("Evaluation complete")

In [ ]:
# ===================================================================
# Cell 15: Confidence Threshold Tuning for False Positive Reduction
# ===================================================================

print("Tuning confidence thresholds to minimize false positives...\n")

# For each class, find optimal threshold that maximizes precision
# while maintaining acceptable recall

optimal_thresholds = {}
threshold_range = np.arange(0.5, 0.99, 0.05)

print("Finding optimal thresholds for each class...")
print("Target: Precision >= 0.95 (max 5% false positive rate)\n")

# Get max probability for each prediction
max_proba = np.max(y_pred_proba, axis=1)

# Test different global thresholds
threshold_results = []

for threshold in threshold_range:
    # Apply threshold: Only predict attack if confidence > threshold
    y_pred_threshold = y_pred.copy()
    
    # If confidence below threshold, predict as benign (class 0)
    low_confidence_mask = max_proba < threshold
    y_pred_threshold[low_confidence_mask] = 0  # Assuming class 0 is benign
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred_threshold)
    prec = precision_score(y_test, y_pred_threshold, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred_threshold, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred_threshold, average='weighted', zero_division=0)
    
    threshold_results.append({
        'threshold': threshold,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1
    })
    
    print(f"Threshold: {threshold:.2f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")

# Convert to DataFrame for analysis
threshold_df = pd.DataFrame(threshold_results)

# Find optimal threshold (highest precision with recall >= 0.85)
valid_thresholds = threshold_df[threshold_df['recall'] >= 0.85]

if len(valid_thresholds) > 0:
    optimal_row = valid_thresholds.loc[valid_thresholds['precision'].idxmax()]
    optimal_threshold = optimal_row['threshold']
else:
    # If no threshold meets recall requirement, choose best F1
    optimal_row = threshold_df.loc[threshold_df['f1'].idxmax()]
    optimal_threshold = optimal_row['threshold']

print(f"\n{'='*60}")
print("Optimal Confidence Threshold")
print(f"{'='*60}")
print(f"Threshold: {optimal_threshold:.2f}")
print(f"Precision: {optimal_row['precision']:.4f}")
print(f"Recall:    {optimal_row['recall']:.4f}")
print(f"F1 Score:  {optimal_row['f1']:.4f}")
print(f"{'='*60}\n")

# Save optimal threshold
threshold_config = {
    'optimal_threshold': float(optimal_threshold),
    'precision': float(optimal_row['precision']),
    'recall': float(optimal_row['recall']),
    'f1': float(optimal_row['f1'])
}

import json
with open(OUTPUT_DIR / 'optimal_threshold.json', 'w') as f:
    json.dump(threshold_config, f, indent=2)

print(f"Optimal threshold saved to: {OUTPUT_DIR / 'optimal_threshold.json'}")

# Plot threshold tuning curve
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(threshold_df['threshold'], threshold_df['precision'], label='Precision', marker='o')
ax.plot(threshold_df['threshold'], threshold_df['recall'], label='Recall', marker='s')
ax.plot(threshold_df['threshold'], threshold_df['f1'], label='F1 Score', marker='^')
ax.axvline(optimal_threshold, color='red', linestyle='--', label=f'Optimal ({optimal_threshold:.2f})')
ax.set_xlabel('Confidence Threshold')
ax.set_ylabel('Score')
ax.set_title('Threshold Tuning for False Positive Reduction')
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'threshold_tuning.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ===================================================================
# Cell 16: Train Traditional ML Model (Random Forest) with Calibration
# ===================================================================

print("Training Random Forest classifier...\n")
print("Random Forest is robust and less prone to overfitting than deep learning")
print("This provides a good ensemble with the neural network\n")

# Train Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight='balanced',  # Handle class imbalance
    random_state=RANDOM_SEED,
    n_jobs=-1,
    verbose=1
)

print("Training Random Forest...")
rf_model.fit(X_train_scaled, y_train)

print("\nCalibrating Random Forest probabilities...")
# Calibrate probabilities to improve confidence estimates
# This reduces false positives by making probability estimates more reliable
rf_calibrated = CalibratedClassifierCV(
    rf_model, 
    method='sigmoid',  # Platt scaling
    cv='prefit'  # Use pre-fitted model
)

rf_calibrated.fit(X_val_scaled, y_val)

print("\nEvaluating Random Forest on test set...")
y_pred_rf = rf_calibrated.predict(X_test_scaled)
y_pred_rf_proba = rf_calibrated.predict_proba(X_test_scaled)

# Calculate metrics
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_precision = precision_score(y_test, y_pred_rf, average='weighted', zero_division=0)
rf_recall = recall_score(y_test, y_pred_rf, average='weighted', zero_division=0)
rf_f1 = f1_score(y_test, y_pred_rf, average='weighted', zero_division=0)

print(f"\n{'='*60}")
print("Random Forest Performance")
print(f"{'='*60}")
print(f"Accuracy:  {rf_accuracy:.4f}")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall:    {rf_recall:.4f}")
print(f"F1 Score:  {rf_f1:.4f}")
print(f"{'='*60}\n")

# Save Random Forest model
rf_model_path = OUTPUT_DIR / 'random_forest_calibrated.pkl'
joblib.dump(rf_calibrated, rf_model_path)
print(f"Random Forest model saved to: {rf_model_path}")

clear_memory()
print_memory_usage("After Random Forest training")

In [ ]:
# ===================================================================
# Cell 17: Ensemble Predictions for Maximum Accuracy
# ===================================================================

print("Creating ensemble predictions...\n")
print("Combining Deep Learning + Random Forest predictions")
print("This reduces false positives by requiring agreement between models\n")

# Strategy 1: Weighted average of probabilities
# Give more weight to the model with higher precision
dl_weight = 0.6 if precision > rf_precision else 0.4
rf_weight = 1.0 - dl_weight

print(f"Ensemble weights:")
print(f"  Deep Learning: {dl_weight:.2f}")
print(f"  Random Forest: {rf_weight:.2f}\n")

# Weighted ensemble
y_pred_ensemble_proba = (dl_weight * y_pred_proba) + (rf_weight * y_pred_rf_proba)
y_pred_ensemble = np.argmax(y_pred_ensemble_proba, axis=1)

# Apply optimal threshold
max_ensemble_proba = np.max(y_pred_ensemble_proba, axis=1)
low_confidence_mask = max_ensemble_proba < optimal_threshold
y_pred_ensemble[low_confidence_mask] = 0  # Default to benign

# Calculate ensemble metrics
ensemble_accuracy = accuracy_score(y_test, y_pred_ensemble)
ensemble_precision = precision_score(y_test, y_pred_ensemble, average='weighted', zero_division=0)
ensemble_recall = recall_score(y_test, y_pred_ensemble, average='weighted', zero_division=0)
ensemble_f1 = f1_score(y_test, y_pred_ensemble, average='weighted', zero_division=0)

print(f"\n{'='*60}")
print("Ensemble Performance (with threshold)")
print(f"{'='*60}")
print(f"Accuracy:  {ensemble_accuracy:.4f}")
print(f"Precision: {ensemble_precision:.4f} ⭐ (Lower false positives!)")
print(f"Recall:    {ensemble_recall:.4f}")
print(f"F1 Score:  {ensemble_f1:.4f}")
print(f"{'='*60}\n")

# Compare all models
comparison_df = pd.DataFrame([
    {'Model': 'Deep Learning', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1},
    {'Model': 'Random Forest', 'Accuracy': rf_accuracy, 'Precision': rf_precision, 'Recall': rf_recall, 'F1': rf_f1},
    {'Model': 'Ensemble', 'Accuracy': ensemble_accuracy, 'Precision': ensemble_precision, 'Recall': ensemble_recall, 'F1': ensemble_f1}
])

print("Model Comparison:")
print(comparison_df.to_string(index=False))
print("\nNote: Higher Precision = Fewer False Positives")

# Save comparison
comparison_df.to_csv(OUTPUT_DIR / 'model_comparison.csv', index=False)

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(comparison_df))
width = 0.2

ax.bar(x - 1.5*width, comparison_df['Accuracy'], width, label='Accuracy')
ax.bar(x - 0.5*width, comparison_df['Precision'], width, label='Precision')
ax.bar(x + 0.5*width, comparison_df['Recall'], width, label='Recall')
ax.bar(x + 1.5*width, comparison_df['F1'], width, label='F1 Score')

ax.set_xlabel('Model')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Model'])
ax.legend()
ax.grid(True, axis='y')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ===================================================================
# Cell 18: Save All Models and Artifacts
# ===================================================================

print("Saving all models and artifacts...\n")

# Save deep learning model
dl_model_path = OUTPUT_DIR / 'dl_model_retrained_fp_optimized.keras'
model.save(dl_model_path)
print(f"✓ Deep Learning model saved: {dl_model_path}")

# Save feature names (important for production deployment)
feature_names = X_train.columns.tolist() if hasattr(X_train, 'columns') else [f'feature_{i}' for i in range(X_train_scaled.shape[1])]
feature_info = {
    'feature_names': feature_names,
    'n_features': len(feature_names)
}
with open(OUTPUT_DIR / 'feature_info.json', 'w') as f:
    json.dump(feature_info, f, indent=2)
print(f"✓ Feature info saved: {OUTPUT_DIR / 'feature_info.json'}")

# Save class mapping
class_mapping = {int(k): str(v) for k, v in enumerate(sorted(y_labels.unique()))}
with open(OUTPUT_DIR / 'class_mapping.json', 'w') as f:
    json.dump(class_mapping, f, indent=2)
print(f"✓ Class mapping saved: {OUTPUT_DIR / 'class_mapping.json'}")

# Save training configuration
training_config = {
    'timestamp': datetime.now().isoformat(),
    'dataset': {
        'total_samples': int(df.shape[0]),
        'train_samples': int(len(y_train)),
        'val_samples': int(len(y_val)),
        'test_samples': int(len(y_test)),
        'n_features': int(n_features),
        'n_classes': int(n_classes)
    },
    'performance': {
        'deep_learning': {
            'accuracy': float(accuracy),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1)
        },
        'random_forest': {
            'accuracy': float(rf_accuracy),
            'precision': float(rf_precision),
            'recall': float(rf_recall),
            'f1': float(rf_f1)
        },
        'ensemble': {
            'accuracy': float(ensemble_accuracy),
            'precision': float(ensemble_precision),
            'recall': float(ensemble_recall),
            'f1': float(ensemble_f1)
        }
    },
    'threshold': threshold_config,
    'ensemble_weights': {
        'deep_learning': float(dl_weight),
        'random_forest': float(rf_weight)
    }
}

with open(OUTPUT_DIR / 'training_config.json', 'w') as f:
    json.dump(training_config, f, indent=2)
print(f"✓ Training config saved: {OUTPUT_DIR / 'training_config.json'}")

print(f"\n{'='*60}")
print("All models and artifacts saved successfully!")
print(f"{'='*60}")
print(f"\nOutput directory: {OUTPUT_DIR}")
print("\nFiles saved:")
for file in sorted(OUTPUT_DIR.glob('*')):
    if file.is_file():
        size_mb = file.stat().st_size / 1024**2
        print(f"  - {file.name} ({size_mb:.2f} MB)")

print_memory_usage("Final")

In [ ]:
# ===================================================================
# Cell 19: Deployment Instructions and Summary
# ===================================================================

print("""\n
╔════════════════════════════════════════════════════════════════════════════╗
║                   MODEL TRAINING COMPLETE!                                 ║
╚════════════════════════════════════════════════════════════════════════════╝

📊 PERFORMANCE SUMMARY
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

print(f"Deep Learning Model:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f} (False Positive Rate: {(1-precision)*100:.2f}%)")
print(f"  Recall:    {recall:.4f}")
print(f"  F1 Score:  {f1:.4f}")

print(f"\nRandom Forest Model:")
print(f"  Accuracy:  {rf_accuracy:.4f}")
print(f"  Precision: {rf_precision:.4f} (False Positive Rate: {(1-rf_precision)*100:.2f}%)")
print(f"  Recall:    {rf_recall:.4f}")
print(f"  F1 Score:  {rf_f1:.4f}")

print(f"\n⭐ Ensemble Model (RECOMMENDED):")
print(f"  Accuracy:  {ensemble_accuracy:.4f}")
print(f"  Precision: {ensemble_precision:.4f} (False Positive Rate: {(1-ensemble_precision)*100:.2f}%)")
print(f"  Recall:    {ensemble_recall:.4f}")
print(f"  F1 Score:  {ensemble_f1:.4f}")

print(f"\nOptimal Confidence Threshold: {optimal_threshold:.2f}")
print(f"  - Predictions below this threshold are classified as BENIGN")
print(f"  - This reduces false positives while maintaining high recall")

print("""\n
📦 DEPLOYMENT FILES
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Required files for deployment:

1. dl_model_retrained_fp_optimized.keras  - Deep learning model
2. random_forest_calibrated.pkl           - Random forest model
3. scaler_standard_retrained.pkl          - Feature scaler
4. optimal_threshold.json                 - Confidence threshold config
5. feature_info.json                      - Feature names and count
6. class_mapping.json                     - Label ID to class name mapping
7. training_config.json                   - Complete training configuration


🚀 INTEGRATION STEPS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Download all files from Google Drive to your local project:
   
   from google.colab import files
   import shutil
   
   # Create archive
   shutil.make_archive('trained_models', 'zip', str(OUTPUT_DIR))
   files.download('trained_models.zip')

2. Extract files to your project's trained_models/ directory

3. Update your config.yaml:
   
   ml:
     ML_MODEL_PATH: trained_models/random_forest_calibrated.pkl
     DL_MODEL_PATH: trained_models/dl_model_retrained_fp_optimized.keras
     SCALER_PATH: trained_models/scaler_standard_retrained.pkl
   
   detection:
     mode: 'threshold'
     confidence_threshold: """ + f"{optimal_threshold:.2f}" + """  # Optimized threshold

4. Restart your detection system to load new models


💡 FALSE POSITIVE REDUCTION TECHNIQUES USED
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✓ Correlation-based feature removal (reduced redundant features)
✓ Focal loss function (focus on hard examples, not easy benign traffic)
✓ Dropout regularization (prevents overfitting)
✓ Hybrid SMOTE + under-sampling (balanced classes without over-representation)
✓ Probability calibration (reliable confidence scores)
✓ Confidence threshold tuning (optimal precision-recall tradeoff)
✓ Model ensemble (requires agreement between DL and RF)
✓ Precision-focused metrics (monitored during training)


⚠️  MONITORING RECOMMENDATIONS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Monitor false positive rate in production
2. If false positives are still too high:
   - Increase confidence_threshold (e.g., 0.85 → 0.90 → 0.95)
   - Enable localhost filtering (filter_localhost: true)
   - Increase min_packet_threshold (e.g., 50 → 100 → 200)

3. If missing real attacks (false negatives):
   - Decrease confidence_threshold (e.g., 0.85 → 0.75)
   - Review anomaly detection settings

4. Collect feedback on false positives and retrain periodically


✅ Training complete! Your models are ready for deployment.
""")

# Show final memory usage
print_memory_usage("Final")

In [ ]:
# ===================================================================
# Cell 20: Download Models (Optional - Run if on Colab)
# ===================================================================

if IN_COLAB:
    print("Creating archive for download...")
    
    import shutil
    from google.colab import files
    
    # Create zip archive
    archive_path = '/content/trained_models_optimized'
    shutil.make_archive(archive_path, 'zip', str(OUTPUT_DIR))
    
    print(f"Archive created: {archive_path}.zip")
    print(f"Size: {os.path.getsize(archive_path + '.zip') / 1024**2:.2f} MB")
    
    print("\nDownloading...")
    files.download(archive_path + '.zip')
    
    print("\n✓ Download complete!")
    print("Extract this file to your local project's trained_models/ directory")
else:
    print("Not running on Colab - models are already saved locally")
    print(f"Location: {OUTPUT_DIR}")